## RealSense AutoNav

Test AutoNav with RealSense data

In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import cv2 as cv
import plotly.graph_objects as go
from collections import defaultdict
from sklearn.linear_model import RANSACRegressor

from rover_nerf.autonav import AutoNav, arc
from rover_nerf.utils import pc_plot_trace
from rover_nerf.autonav_utils import local_to_global, depth_to_points, compute_slope_and_roughness, estimate_hessian_trace

# autoreload
%load_ext autoreload
%autoreload 2

# line profiler
%load_ext line_profiler

In [ ]:
points = np.load('../data/realsense/pointcloud_0.npy')
# Downsample points
points = points[::10,:]

In [ ]:
# Transform
transform = np.array([[0,-1, 0], 
                      [0, 0,-1], 
                      [1, 0, 0]])
points = points @ transform

In [ ]:
# Filter out points beyond a certain distance
max_depth = 2.0  # meters
points = points[points[:,0] < max_depth,:]
max_height = -0.1  # meters
points = points[points[:,2] < max_height,:]

In [ ]:
trace = pc_plot_trace(points, color=points[:,2])
fig = go.Figure(data=[trace])
fig.update_layout(width=1600, height=900, scene_aspectmode='data')
fig.show()

In [ ]:
autonav = AutoNav(goal=np.array([0,0]), cmap_resolution=0.1, max_depth=3)

In [ ]:
pose = np.array([0,0,0])
cost_vals = autonav.update_costmap(pose, None, points)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
im = autonav.plot_costmap(ax)
plt.colorbar(im, fraction=0.04, aspect=12)

In [ ]:
cmap_resolution = 0.1  # m
max_depth = 3  # m
W = int(max_depth / cmap_resolution)
cmap_dims = [W + 1, 2*W + 1]  # m
cmap_center = [W, W]
costmap = np.zeros(cmap_dims)

In [ ]:
centroid = np.mean(points[:,:3], axis=0)

bins = defaultdict(list)
scale = cmap_resolution

x_indices = cmap_center[0] - (points[:, 0] / scale).astype(int)
y_indices = cmap_center[1] + (points[:, 1] / scale).astype(int)
print("X and Y indices: ", x_indices, y_indices, len(x_indices), len(y_indices))

In [ ]:
adjusted_points = points[:,:3] - centroid
print("Shape of adjusted points: ", adjusted_points.shape)

In [ ]:
for x_idx, y_idx, point in zip(x_indices, y_indices, adjusted_points):
    bins[(x_idx, y_idx)].append(tuple(point))

cost_vals = []                          
for k, v in bins.items():

    bin_pts = np.array(v)
    print("Bin pts: ", len(bin_pts))
    
    if len(bin_pts) > 10:
        dem = {}
        dem_resolution = 0.1
        min_x, min_y = np.inf, np.inf
        max_x, max_y = -np.inf, -np.inf
        for i, (x, y, z) in enumerate(bin_pts):
            x = int(x / dem_resolution)
            y = int(y / dem_resolution)
            if (x, y) not in dem:
                dem[(x, y)] = z
            else:
                dem[(x, y)] = max((dem[(x, y)], z))

        xy_vals = np.array(list(dem.keys()))
        z_vals = np.array(list(dem.values()))
        #print("xy_vals: ", xy_vals.shape)

        ransac = RANSACRegressor(max_trials=10, residual_threshold=0.01)
        # self.ransac.fit(xy_vals, z_vals)
        try:
            print("Have fitted RANSAC")
            ransac.fit(xy_vals, z_vals)
        except:
            print("RANSAC failed")
            print("xy_vals: ", xy_vals.shape)
            print("z_vals: ", z_vals.shape)
            continue
        a, b = ransac.estimator_.coef_
        # a, b = 0, 0

        z_pred = ransac.estimator_.predict(xy_vals)
        # z_pred = np.zeros(len(z_vals))
        loss_vals = z_pred - z_vals

        roughness = estimate_hessian_trace(np.hstack((xy_vals, loss_vals[:, None])))

        n = np.array([a, b, -1])
        n = n / np.linalg.norm(n)
        if n[2] < 0:
            n = -n
        slope = np.abs(np.arccos(np.dot(n, np.array([0, 0, 1]))))
        cost = 1.0 * slope + 20.0 * roughness
        
        costmap[k] = cost  # update costmap

        # Convert local coordinates to global coordinates
        pose = np.array([0.0, 0.0, 0.0]) 
        local_x = cmap_center[0] - k[0]
        local_y = cmap_center[1] - k[1]
        global_x = local_x * np.cos(pose[2]) - local_y * np.sin(pose[2]) + pose[0]
        global_y = local_x * np.sin(pose[2]) + local_y * np.cos(pose[2]) + pose[1]
        cost_vals.append([global_x, global_y, cost])